In [22]:
# Instala pandas si no está instalado y verifica la instalación
try:
    import pandas as pd
except ImportError:
    import sys
    !{sys.executable} -m pip install pandas
    import pandas as pd
print('pandas versión:', pd.__version__)

pandas versión: 2.3.3


# Decodificación masiva de imágenes JPEG2000

A continuación se recorre todas las imágenes en `tests/test_images/JPEG2000`, se intenta decodificarlas usando la librería `jpegexp-rs` y un decodificador estándar (imagecodecs/OpenJPEG), y se documentan los resultados para cada archivo en una tabla.

In [23]:
import os
import subprocess
import imagecodecs
import pandas as pd
from IPython.display import display, HTML

# Ruta absoluta a las imágenes de prueba (ajusta aquí si es necesario)
TEST_IMAGES_DIR = os.path.abspath("./test_images/JPEG2000")

# Ruta correcta al binario de jpegexp-rs
if os.name == "nt":
    bin_path = os.path.abspath(os.path.join("./../", "target","release", "jpegexp.exe"))
else:
    bin_path = os.path.abspath(os.path.join("./../", "target", "release", "jpegexp"))
JPEGEXP_BIN = bin_path

print(f"Directorio de imágenes: {TEST_IMAGES_DIR}")
print(f"Binario jpegexp-rs: {JPEGEXP_BIN}")
print(f"¿Existe el binario?: {os.path.exists(JPEGEXP_BIN)}")

# Buscar todos los archivos, incluyendo subdirectorios
image_files = []
for root, dirs, files in os.walk(TEST_IMAGES_DIR):
    for f in files:
        image_files.append(os.path.join(root, f))

print(f"Total de archivos encontrados (recursivo): {len(image_files)}")

results = []

if not os.path.exists(JPEGEXP_BIN):
    print(f"ERROR: El binario jpegexp no existe en {JPEGEXP_BIN}. Compílalo con 'cargo build --release' en la raíz del proyecto.")
else:
    if not image_files:
        print(f"No se encontraron imágenes en {TEST_IMAGES_DIR} (ruta absoluta: {os.path.abspath(TEST_IMAGES_DIR)})")
    else:
        print(f"Se encontraron {len(image_files)} archivos en {TEST_IMAGES_DIR}")
        for img_path in image_files:
            fname = os.path.relpath(img_path, TEST_IMAGES_DIR)
            entry = {"file": fname}
            # --- Decodificación con jpegexp-rs ---
            try:
                out_raw = f"{img_path}.jpegexp.raw"
                cmd = [JPEGEXP_BIN, "decode", "-i", img_path, "-o", out_raw, "-f", "raw"]
                proc = subprocess.run(cmd, capture_output=True, text=True, timeout=10)
                if proc.returncode != 0 and 'panicked' in proc.stderr:
                    entry["jpegexp_rs"] = f"PANIC: {proc.stderr.strip()}"
                else:
                    entry["jpegexp_rs"] = "OK" if proc.returncode == 0 else f"ERROR: {proc.stderr.strip()}"
            except Exception as e:
                entry["jpegexp_rs"] = f"EXCEPTION: {e}"
            # --- Decodificación con imagecodecs (OpenJPEG) ---
            try:
                with open(img_path, "rb") as f:
                    arr = imagecodecs.jpeg2k_decode(f.read())
                entry["imagecodecs"] = "OK"
            except Exception as e:
                entry["imagecodecs"] = f"ERROR: {e}"
            results.append(entry)
        df = pd.DataFrame(results)
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_colwidth', None)
        display(HTML(df.to_html(index=False)))

Directorio de imágenes: c:\Users\aroja\CODE\jpegexp-rs\tests\test_images\JPEG2000
Binario jpegexp-rs: c:\Users\aroja\CODE\jpegexp-rs\target\release\jpegexp.exe
¿Existe el binario?: True
Total de archivos encontrados (recursivo): 134
Se encontraron 134 archivos en c:\Users\aroja\CODE\jpegexp-rs\tests\test_images\JPEG2000


file,jpegexp_rs,imagecodecs
aware.jp2,ERROR: Error: Invalid data,OK
bitwiser-codestreamheader-corrupted-xsiz-10918.jp2,PANIC: thread 'main' (22348) panicked at src\jpeg2000\dwt.rs:317:58:\nindex out of bounds: the len is 1 but the index is 1\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace,ERROR: opj_read_header failed
bitwiser-codestreamheader-corrupted-xsiz-10928.jp2,PANIC: thread 'main' (15964) panicked at src\jpeg2000\dwt.rs:317:58:\nindex out of bounds: the len is 1 but the index is 1\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace,ERROR: opj_read_header failed
bitwiser-codestreamheader-corrupted-xsiz-10937.jp2,PANIC: thread 'main' (34756) panicked at src\jpeg2000\dwt.rs:317:58:\nindex out of bounds: the len is 1 but the index is 1\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace,ERROR: opj_read_header failed
bitwiser-codestreamheader-corrupted-xsiz-10946.jp2,PANIC: thread 'main' (6928) panicked at src\jpeg2000\dwt.rs:317:58:\nindex out of bounds: the len is 1 but the index is 1\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace,ERROR: opj_read_header failed
bitwiser-codestreamheader-corrupted-xsiz-10955.jp2,PANIC: thread 'main' (32492) panicked at src\jpeg2000\dwt.rs:317:58:\nindex out of bounds: the len is 1 but the index is 1\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace,ERROR: opj_read_header failed
bitwiser-codestreamheader-corrupted-ysiz-11208.jp2,PANIC: thread 'main' (17972) panicked at src\jpeg2000\dwt.rs:317:58:\nindex out of bounds: the len is 1 but the index is 1\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace,ERROR: opj_read_header failed
bitwiser-codestreamheader-corrupted-ysiz-11218.jp2,PANIC: thread 'main' (19448) panicked at src\jpeg2000\dwt.rs:317:58:\nindex out of bounds: the len is 1 but the index is 1\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace,ERROR: opj_read_header failed
bitwiser-codestreamheader-corrupted-ysiz-11227.jp2,PANIC: thread 'main' (14900) panicked at src\jpeg2000\dwt.rs:317:58:\nindex out of bounds: the len is 1 but the index is 1\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace,ERROR: opj_read_header failed
bitwiser-codestreamheader-corrupted-ysiz-11238.jp2,PANIC: thread 'main' (24688) panicked at src\jpeg2000\dwt.rs:317:58:\nindex out of bounds: the len is 1 but the index is 1\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace,ERROR: opj_read_header failed


## Instrucciones
- Ejecuta la celda anterior para obtener una tabla con los resultados de decodificación de cada archivo.
- La columna `jpegexp_rs` muestra el resultado usando la librería Rust.
- La columna `imagecodecs` muestra el resultado usando el decodificador estándar de Python (OpenJPEG).
- Los errores o excepciones se documentan en la tabla para análisis posterior.